# Preliminary Data Annotation

We want positive/negative examples annotated with a series of linguistic metrics (coherence, fluency) both at the utterance level and at the dialogue level (< 5 turns). 

- Positive examples will be taken from the [BabyLM (Switchboard)](https://huggingface.co/datasets/hhoangphuoc/switchboard) dataset.
- Negative examples will be taken from BabyLlama outputs.

Corpus size: no more than 30M tokens.

## 2. Data Processing

### 2.1 BabyLM (Switchboard) Dataset

In [5]:
# Switchboard Data Processing for Lexical Metrics

import pandas as pd
import os

# === Load and Clean Data ===
with open("/home/rooein/babylm-interaction/baseline/data/train_100M/switchboard.train", "r") as f:
    lines = [line.strip() for line in f if line.strip() and not line.startswith("----")]

# Parse speaker and text
data = []
for line in lines:
    if "\t" in line:
        speaker, text = line.split("\t", 1)
        data.append((speaker.strip(), text.strip()))

# Create DataFrame
df = pd.DataFrame(data, columns=["speaker", "text"])

# === Dialog-Level Chunking (5-turn blocks like ABABA or BABAB) ===
dialogs = []
current_dialog = []
last_speaker = None

for speaker, text in zip(df["speaker"], df["text"]):
    if not current_dialog:
        current_dialog.append([speaker, [text]])
    elif speaker == last_speaker:
        current_dialog[-1][1].append(text)
    else:
        current_dialog.append([speaker, [text]])

    last_speaker = speaker

    if len(current_dialog) == 5:
        dialog_text = [f"{turn[0]}: {' '.join(turn[1])}" for turn in current_dialog]
        dialogs.append(dialog_text)
        current_dialog = []
        last_speaker = None

# Convert to DataFrame
dialog_df = pd.DataFrame(dialogs, columns=[f"turn_{i+1}" for i in range(5)])

# === Turn-Level Extraction ===
turns = []
for dialog in dialogs:
    for turn in dialog:
        turns.append(turn)

turn_df = pd.DataFrame(turns, columns=["turn_text"])

# === Speaker-Level Aggregation ===
speaker_chunks = []

for dialog in dialogs:
    a_text, b_text = [], []
    for turn_text in dialog:
        if turn_text.startswith("A:"):
            a_text.append(turn_text[2:].strip())
        elif turn_text.startswith("B:"):
            b_text.append(turn_text[2:].strip())
    speaker_chunks.append({
        "A_text": " ".join(a_text),
        "B_text": " ".join(b_text),
    })

speaker_df = pd.DataFrame(speaker_chunks)

# === Save DataFrames ===
dialog_df.to_csv("switchboard_dialog_level.csv", index=False)
# turn_df.to_csv("switchboard_turn_level.csv", index=False)
# speaker_df.to_csv("switchboard_speaker_level.csv", index=False)

print("Saved dialog level files.")

# === Export Each Row to Text Files ===
os.makedirs("./switchboard/turn_level_texts/A", exist_ok=True)
os.makedirs("./switchboard/turn_level_texts/B", exist_ok=True)

for i, row in enumerate(turn_df["turn_text"]):
    speaker = row.strip()[0] if row.strip() else "Unknown"
    if speaker in ["A", "B"]:
        with open(f"./switchboard/turn_level_texts/{speaker}/{speaker}_{i:05d}.txt", "w") as f:
            f.write(row)

os.makedirs("./switchboard/speaker_level_texts/A", exist_ok=True)
os.makedirs("./switchboard/speaker_level_texts/B", exist_ok=True)

for i, row in speaker_df.iterrows():
    with open(f"./switchboard/speaker_level_texts/A/A_{i:05d}.txt", "w") as fa:
        fa.write(row["A_text"])
    with open(f"./switchboard/speaker_level_texts/B/B_{i:05d}.txt", "w") as fb:
        fb.write(row["B_text"])

# prepare text inputs based on dialogs
os.makedirs("./switchboard/dialog_level_texts", exist_ok=True)

for i, row in dialog_df.iterrows():
    full_dialog = "\n".join(row.values)
    with open(f"./switchboard/dialog_level_texts/dialog_{i:05d}.txt", "w") as f:
        f.write(full_dialog)

print("Exported text files to ./switchboard/turn_level_texts/, ./switchboard/speaker_level_texts/, and ./switchboard/dialog_level_texts/")


Saved dialog level files.
Exported text files to ./switchboard/turn_level_texts/, ./switchboard/speaker_level_texts/, and ./switchboard/dialog_level_texts/


## 3. Metrics

We discussed Fluency and Coherence as the two important things we want to annotate.

### 3.1.1 Loading TAACO Outputs 

TAACO accepts inputs as `.txt` files. To prepare inputs for TAACO, we first chunk the dataset into individual turns using the `switchboard_data`, and save each dialog separately as `.txt` files.

Next, we run TAACO metrics on directory of text files by executing `test_donya.py` located in the TAACO directory. `test_donya.py` reads a list of directory of text files from `taaco_config.json` and TAACO outputs are then saved to the directory `switchboard_taaco_results`.



### 3.1.2 Filtering TAACO Outputs 
We subsequently filter the TAACO outputs, retaining only the following selected metrics:

- `noun_ttr`
- `verb_ttr`
- `adj_ttr`
- `lemma_ttr`
- `bigram_lemma_ttr`
- `trigram_lemma_ttr`
- `adjacent_overlap_all_sent`
- `lda_1_all_sent`
- `repeated_content_lemmas`
- `repeated_content_and_pronoun_lemmas`




In [30]:
import pandas as pd
import os
import re

# Paths
csv_file_path = '/home/rooein/babylm-interaction/baseline/data/switchboard_taaco_results/dialog_level_taaco_results.csv'
texts_folder_path = '/home/rooein/babylm-interaction/baseline/data/switchboard_data/dialog_level_texts'
output_csv_path = '/home/rooein/babylm-interaction/baseline/data/switchboard_taaco_results/dialog_level_taaco_results_filtered.csv'

# Step 1: Load CSV
metrics_df = pd.read_csv(csv_file_path)

# Step 2: Load and map text files to filenames
def load_text(filename, folder):
    filepath = os.path.join(folder, filename)
    with open(filepath, 'r', encoding='utf-8') as file:
        return file.read().strip()

# Add text column based on Filename
metrics_df['text'] = metrics_df['Filename'].apply(lambda fname: load_text(fname, texts_folder_path))

# Step 3: Compute number of words without A:: and B:: in the texts and rank texts
def clean_text(text):
    return re.sub(r'\b[AB]::', '', text)

metrics_df['cleaned_text'] = metrics_df['text'].apply(clean_text)
metrics_df['num_words'] = metrics_df['cleaned_text'].apply(lambda x: len(x.split()))

print(f"Total tokens before ranking: {metrics_df['num_words'].sum()}")

metrics_df.sort_values(by='num_words', ascending=False, inplace=True)

# Step 4: Select data until reaching approximately 30 million tokens
max_tokens = 30_000_000
token_count = 0
selected_rows = []

for idx, row in metrics_df.iterrows():
    if token_count + row['num_words'] <= max_tokens:
        selected_rows.append(row)
        token_count += row['num_words']
    else:
        break

selected_df = pd.DataFrame(selected_rows)

print(f"Total words after ranking and selection: {selected_df['num_words'].sum()}")

# Step 5: Save the final DataFrame
selected_df.to_csv(output_csv_path, index=False)

# Display the first few rows
print(selected_df.head())


Total tokens before ranking: 1180289
Total words after ranking and selection: 1180289
               Filename  lemma_ttr  lemma_mattr  lexical_density_tokens  \
2341   dialog_02341.txt   0.332795     0.670491                0.387722   
2633   dialog_02633.txt   0.336299     0.726121                0.450178   
8707   dialog_08707.txt   0.321839     0.706173                0.421456   
12658  dialog_12658.txt   0.342697     0.747258                0.430712   
283    dialog_00283.txt   0.355509     0.727546                0.432432   

       lexical_density_types  content_ttr  function_ttr  function_mattr  \
2341                0.728155     0.625000      0.171504        0.462970   
2633                0.761905     0.569170      0.190939        0.470385   
8707                0.720238     0.550000      0.192053        0.497312   
12658               0.666667     0.530435      0.233553        0.542196   
283                 0.736842     0.605769      0.190476        0.480000   

       noun_

In [31]:

# Define TAACO metrics columns
taaco_cols = [
    'noun_ttr', 'verb_ttr', 'adj_ttr', 'lemma_ttr',
    'bigram_lemma_ttr', 'trigram_lemma_ttr', 'adjacent_overlap_all_sent',
    'lda_1_all_sent', 'repeated_content_lemmas',
    'repeated_content_and_pronoun_lemmas'
]

# Create new column with TAACO metrics as a dictionary
selected_df['TAACO_metrics_dialog'] = selected_df[taaco_cols].to_dict(orient='records')

# Keep only the desired columns
df_reformatted = selected_df[['Filename', 'text', 'num_words', 'TAACO_metrics_dialog']]

# Display or save the new DataFrame
print(df_reformatted.head())


               Filename                                               text  \
2341   dialog_02341.txt  B:: And nothing is being done about it. Uh, th...   
2633   dialog_02633.txt  B:: I don't know if any of mine will be intere...   
8707   dialog_08707.txt  A:: I put a stop to some of them as far as the...   
12658  dialog_12658.txt  A:: You know, my neighbors across the street, ...   
283    dialog_00283.txt  A:: Yeah. Have you, do you use a standard, uh,...   

       num_words                               TAACO_metrics_dialog  
2341         593  {'noun_ttr': 0.71, 'verb_ttr': 0.475, 'adj_ttr...  
2633         542  {'noun_ttr': 0.6597938144329897, 'verb_ttr': 0...  
8707         492  {'noun_ttr': 0.68, 'verb_ttr': 0.4634146341463...  
12658        489  {'noun_ttr': 0.6216216216216216, 'verb_ttr': 0...  
283          460  {'noun_ttr': 0.7088607594936709, 'verb_ttr': 0...  


In [32]:
df_reformatted

,Filename,text,num_words,TAACO_metrics_dialog
2341,dialog_02341.txt,"B:: And nothing is being done about it. Uh, th...",593,"{'noun_ttr': 0.71, 'verb_ttr': 0.475, 'adj_ttr..."
2633,dialog_02633.txt,B:: I don't know if any of mine will be intere...,542,"{'noun_ttr': 0.6597938144329897, 'verb_ttr': 0..."
8707,dialog_08707.txt,A:: I put a stop to some of them as far as the...,492,"{'noun_ttr': 0.68, 'verb_ttr': 0.4634146341463..."
12658,dialog_12658.txt,"A:: You know, my neighbors across the street, ...",489,"{'noun_ttr': 0.6216216216216216, 'verb_ttr': 0..."
283,dialog_00283.txt,"A:: Yeah. Have you, do you use a standard, uh,...",460,"{'noun_ttr': 0.7088607594936709, 'verb_ttr': 0..."
...,...,...,...,...
1093,dialog_01093.txt,"A:: Yeah.\nB:: and,\nA:: Uh-huh.\nB:: I don't ...",7,"{'noun_ttr': 0.4, 'verb_ttr': 1.0, 'adj_ttr': ..."
10672,dialog_10672.txt,B:: Uh-huh.\nA:: Small island.\nB:: Yeah.\nA::...,6,"{'noun_ttr': 0.5, 'verb_ttr': 0.0, 'adj_ttr': ..."
6125,dialog_06125.txt,A:: Thanks.\nB:: Thank you.\nA:: Bye.\nB:: Goo...,6,"{'noun_ttr': 0.5, 'verb_ttr': 1.0, 'adj_ttr': ..."
13613,dialog_13613.txt,A:: Okay.\nB:: Thanks.\nA:: Thank you.\nB:: By...,6,"{'noun_ttr': 0.4444444444444444, 'verb_ttr': 1..."


In [33]:
# Step 1: Load speaker-level TAACO results
speaker_A_df = pd.read_csv('/home/rooein/babylm-interaction/baseline/data/switchboard_taaco_results/speaker_level_A_taaco_results.csv')
speaker_B_df = pd.read_csv('/home/rooein/babylm-interaction/baseline/data/switchboard_taaco_results/speaker_level_B_taaco_results.csv')

# Step 2: Extract base filename from 'Filename'
speaker_A_df['BaseFilename'] = speaker_A_df['Filename'].apply(lambda x: x.replace('A_', ''))
speaker_B_df['BaseFilename'] = speaker_B_df['Filename'].apply(lambda x: x.replace('B_', ''))
df_reformatted['BaseFilename'] = df_reformatted['Filename'].str.extract(r'(\d+\.txt)$')

# Step 3: Define TAACO metrics columns
taaco_cols = [
    'noun_ttr', 'verb_ttr', 'adj_ttr', 'lemma_ttr',
    'bigram_lemma_ttr', 'trigram_lemma_ttr', 'adjacent_overlap_all_sent',
    'lda_1_all_sent', 'repeated_content_lemmas',
    'repeated_content_and_pronoun_lemmas'
]

# Step 4: Create dictionaries of metrics
speaker_A_df['TAACO_metrics_speaker_A'] = speaker_A_df[taaco_cols].to_dict(orient='records')
speaker_B_df['TAACO_metrics_speaker_B'] = speaker_B_df[taaco_cols].to_dict(orient='records')

# Step 5: Merge back to main df
df_reformatted = df_reformatted.merge(
    speaker_A_df[['BaseFilename', 'TAACO_metrics_speaker_A']],
    on='BaseFilename', how='left'
)

df_reformatted = df_reformatted.merge(
    speaker_B_df[['BaseFilename', 'TAACO_metrics_speaker_B']],
    on='BaseFilename', how='left'
)

# Step 6: Drop helper column
df_reformatted.drop(columns=['BaseFilename'], inplace=True)

# Step 7: Save updated dataframe
df_reformatted.to_csv('/home/rooein/babylm-interaction/baseline/data/switchboard_taaco_results/dialog_level_taaco_results_filtered_with_speakers.csv', index=False)




/tmp/ipykernel_58849/2340559237.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reformatted['BaseFilename'] = df_reformatted['Filename'].str.extract(r'(\d+\.txt)$')


In [34]:
df_reformatted

,Filename,text,num_words,TAACO_metrics_dialog,TAACO_metrics_speaker_A,TAACO_metrics_speaker_B
0,dialog_02341.txt,"B:: And nothing is being done about it. Uh, th...",593,"{'noun_ttr': 0.71, 'verb_ttr': 0.475, 'adj_ttr...","{'noun_ttr': 0.8166666666666667, 'verb_ttr': 0...","{'noun_ttr': 0.8857142857142857, 'verb_ttr': 0..."
1,dialog_02633.txt,B:: I don't know if any of mine will be intere...,542,"{'noun_ttr': 0.6597938144329897, 'verb_ttr': 0...","{'noun_ttr': 0.7246376811594203, 'verb_ttr': 0...","{'noun_ttr': 0.8695652173913043, 'verb_ttr': 0..."
2,dialog_08707.txt,A:: I put a stop to some of them as far as the...,492,"{'noun_ttr': 0.68, 'verb_ttr': 0.4634146341463...","{'noun_ttr': 0.8695652173913043, 'verb_ttr': 0...","{'noun_ttr': 0.7446808510638298, 'verb_ttr': 0..."
3,dialog_12658.txt,"A:: You know, my neighbors across the street, ...",489,"{'noun_ttr': 0.6216216216216216, 'verb_ttr': 0...","{'noun_ttr': 0.6376811594202898, 'verb_ttr': 0...","{'noun_ttr': 0.0, 'verb_ttr': 1.0, 'adj_ttr': ..."
4,dialog_00283.txt,"A:: Yeah. Have you, do you use a standard, uh,...",460,"{'noun_ttr': 0.7088607594936709, 'verb_ttr': 0...","{'noun_ttr': 0.8235294117647058, 'verb_ttr': 0...","{'noun_ttr': 0.775, 'verb_ttr': 0.479166666666..."
...,...,...,...,...,...,...
17888,dialog_01093.txt,"A:: Yeah.\nB:: and,\nA:: Uh-huh.\nB:: I don't ...",7,"{'noun_ttr': 0.4, 'verb_ttr': 1.0, 'adj_ttr': ...","{'noun_ttr': 0.0, 'verb_ttr': 0.0, 'adj_ttr': ...","{'noun_ttr': 0.0, 'verb_ttr': 1.0, 'adj_ttr': ..."
17889,dialog_10672.txt,B:: Uh-huh.\nA:: Small island.\nB:: Yeah.\nA::...,6,"{'noun_ttr': 0.5, 'verb_ttr': 0.0, 'adj_ttr': ...","{'noun_ttr': 1.0, 'verb_ttr': 0.0, 'adj_ttr': ...","{'noun_ttr': 0.0, 'verb_ttr': 0.0, 'adj_ttr': ..."
17890,dialog_06125.txt,A:: Thanks.\nB:: Thank you.\nA:: Bye.\nB:: Goo...,6,"{'noun_ttr': 0.5, 'verb_ttr': 1.0, 'adj_ttr': ...","{'noun_ttr': 1.0, 'verb_ttr': 0.0, 'adj_ttr': ...","{'noun_ttr': 1.0, 'verb_ttr': 1.0, 'adj_ttr': ..."
17891,dialog_13613.txt,A:: Okay.\nB:: Thanks.\nA:: Thank you.\nB:: By...,6,"{'noun_ttr': 0.4444444444444444, 'verb_ttr': 1...","{'noun_ttr': 1.0, 'verb_ttr': 1.0, 'adj_ttr': ...","{'noun_ttr': 0.6666666666666666, 'verb_ttr': 0..."
